In [1]:
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

Tensorflow version 2.19.0


In [2]:
# 0: 'angry',
# 1: 'happy',
# 2: relaxed',
# 3: 'sad'

import os
driver_image_path = r'C:\Users\U2SR\Desktop\Udemy\tensorflow-cnn\Dog Emotion'
class_names = sorted(os.listdir(driver_image_path))
print(class_names)

['angry', 'happy', 'relaxed', 'sad']


In [ ]:
import cv2
imageee = cv2.imread(r'C:\Users\U2SR\Desktop\Udemy\tensorflow-cnn\Dog Emotion\angry\0aNyXBrmNA7XdefwHvgO2n1rnpqQAp885.jpg')
imageee.shape

(400, 440, 3)

In [8]:
img_size = (200, 200, 3)

images = []
labels = []

for class_name in class_names:
    images_files_folder_name = os.path.join(driver_image_path, class_name)
    if os.path.isdir(images_files_folder_name):
        for image_name in os.listdir(images_files_folder_name):
            if image_name.lower().endswith('.jpg'):
                image_full_path = os.path.join(images_files_folder_name, image_name)
                
                img = cv2.imread(image_full_path)
                if img is None:
                    continue
                
                img_cvt = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img_resized = cv2.resize(img_cvt, img_size[:2])
                
                images.append(img_resized)
                labels.append(class_names.index(class_name))

print(f"Total images : {len(images)}, Total labels : {len(labels)}")

Total images : 3975, Total labels : 3975


In [9]:
import numpy as np
images = np.asarray(images)
labels = np.asarray(labels)

In [10]:
from sklearn.model_selection import train_test_split
X_train , X_val , y_train , y_val = train_test_split(images , labels , test_size = 0.2)

In [11]:
import tensorflow as tf
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomRotation(factor=0.05),             #랜덤 회전
    tf.keras.layers.RandomFlip(mode='horizontal'),           #랜덤 수평 플립
    tf.keras.layers.RandomContrast(factor=0.2),              #랜덤 대비 조정
])

In [12]:
#이미지 크기를 지정된 크기로 조정해주는 함수
def preprocessing_image(image , label):
  image = tf.image.resize(image , img_size[:2])
  return image , label

#이미지에 이미지 증강을 적용해주는 함수
def augment_image(image , label):
  image = data_augmentation(image)
  return image , label

#이미지와 레이블을 tensorflow dataset으로 변환하고, 전처리 및 증강을 적용하여 배치와 프리패칭을 설정하는 함수
def create_tensorflow_dataset(images ,labels , batch_size = 32 , buffer_size = 1000 , augment = False):
  dataset = tf.data.Dataset.from_tensor_slices((images , labels))
  dataset = dataset.map(preprocessing_image , num_parallel_calls = tf.data.AUTOTUNE)
  if augment:
    dataset = dataset.map(augment_image , num_parallel_calls = tf.data.AUTOTUNE)
  dataset = dataset.shuffle(buffer_size)
  dataset = dataset.batch(batch_size)
  dataset = dataset.prefetch(buffer_size = tf.data.AUTOTUNE)
  return dataset

In [15]:
import tensorflow as tf

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU 탐색
    print('Running on TPU:', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
    global_batch_size = 32 * tpu_strategy.num_replicas_in_sync
else:
    global_batch_size = 32

In [16]:
train_dataset = create_tensorflow_dataset(X_train , y_train , batch_size = global_batch_size , buffer_size = len(X_train) , augment = True)
val_dataset = create_tensorflow_dataset(X_val , y_val , batch_size = global_batch_size , buffer_size = len(X_val) , augment = False)

In [17]:
img_size = (200, 200, 3)

with tpu_strategy.scope():
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=img_size),
        tf.keras.layers.Rescaling(scale=1./255),

        tf.keras.layers.Conv2D(16, kernel_size=3, padding='same', activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=2),

        tf.keras.layers.Conv2D(32, kernel_size=3, padding='same', activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=2),

        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(len(class_names), activation='softmax')
    ])

    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

history = model.fit(train_dataset, epochs=100, validation_data=val_dataset, callbacks=[early_stopping_cb])

model.save('my_custom_cnn_model.h5')


NameError: name 'tpu_strategy' is not defined